# Bài Tập Thiết Kế Cơ Sở Dữ Liệu Hệ Thống Quản Lý Bán Hàng Quần Áo

In [1]:
import pymysql
import os
from dotenv import load_dotenv

# Load environment variables from .credential file
load_dotenv("../../w7/.credential")

# Get credentials from environment variables
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cursor = conn.cursor()


In [2]:
cursor.execute("CREATE DATABASE IF NOT EXISTS w9db1;")
cursor.execute("USE w9db1;")

# 1. Tạo bảng Categories
cursor.execute("""
CREATE TABLE IF NOT EXISTS Categories (
    CategoryID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL
);
""")

# 2. Tạo bảng Products
cursor.execute("""
CREATE TABLE IF NOT EXISTS Products (
    ProductID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL,
    CategoryID INT,
    Price DECIMAL(10, 2),
    FOREIGN KEY (CategoryID) REFERENCES Categories(CategoryID)
);
""")

# 3. Tạo bảng Variants
cursor.execute("""
CREATE TABLE IF NOT EXISTS Variants (
    VariantID INT PRIMARY KEY,
    ProductID INT,
    Color VARCHAR(50),
    Size VARCHAR(10),
    Stock INT,
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
""")

# 4. Tạo bảng Customers
cursor.execute("""
CREATE TABLE IF NOT EXISTS Customers (
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(255) NOT NULL,
    Phone VARCHAR(20),
    Address VARCHAR(255),
    CreatedDate DATE
);
""")

# 5. Tạo bảng Orders
cursor.execute("""
CREATE TABLE IF NOT EXISTS Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate DATE,
    Status VARCHAR(50),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);
""")

# 6. Tạo bảng OrderDetails
cursor.execute("""
CREATE TABLE IF NOT EXISTS OrderDetails (
    OrderID INT,
    VariantID INT,
    Quantity INT,
    Price DECIMAL(10, 2),
    PRIMARY KEY (OrderID, VariantID),
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (VariantID) REFERENCES Variants(VariantID)
);
""")

# Thêm dữ liệu vào bảng Categories
cursor.execute("""
INSERT INTO Categories (CategoryID, Name) VALUES
(1, 'Áo'),
(2, 'Quần'),
(3, 'Váy'),
(4, 'Phụ kiện')
;
""")

# Thêm dữ liệu vào bảng Products
cursor.execute("""
INSERT INTO Products (ProductID, Name, CategoryID, Price) VALUES
(101, 'Áo sơ mi nam', 1, 300000),
(102, 'Quần jeans', 2, 500000),
(103, 'Váy xòe', 3, 450000)
;
""")

# Thêm dữ liệu vào bảng Variants
cursor.execute("""
INSERT INTO Variants (VariantID, ProductID, Color, Size, Stock) VALUES
(1, 101, 'Trắng', 'M', 20),
(2, 101, 'Trắng', 'L', 15),
(3, 102, 'Xanh', 'M', 10),
(4, 103, 'Đỏ', 'S', 5);
""")

# Thêm dữ liệu vào bảng Customers
cursor.execute("""
INSERT INTO Customers (CustomerID, Name, Phone, Address, CreatedDate) VALUES
(1, 'Trần An', '0909123456', 'Hà Nội', '2024-01-10'),
(2, 'Lê Bình', '0909223456', 'TP.HCM', '2024-02-05');
""")

# Thêm dữ liệu vào bảng Orders
cursor.execute("""
INSERT INTO Orders (OrderID, CustomerID, OrderDate, Status) VALUES
(1, 1, '2024-03-01', 'Đã giao'),
(2, 2, '2024-03-02', 'Đã đặt');
""")

# Thêm dữ liệu vào bảng OrderDetails
cursor.execute("""
INSERT INTO OrderDetails (OrderID, VariantID, Quantity, Price) VALUES
(1, 1, 2, 300000),
(1, 3, 1, 500000),
(2, 4, 1, 450000);
""")

# Commit the changes
conn.commit()

In [3]:
# 3. Liệt kê tất cả các đơn hàng, kèm theo tên khách hàng và tổng tiền của mỗi đơn hàng.

cursor.execute("""
SELECT o.orderid, c.name, od.price
FROM customers c
INNER JOIN orders o
    ON c.customerid = o.customerid
INNER JOIN orderdetails od
    ON o.orderid = od.orderid;
""")

data = cursor.fetchall()
print(data)

((1, 'Trần An', Decimal('300000.00')), (1, 'Trần An', Decimal('500000.00')), (2, 'Lê Bình', Decimal('450000.00')))


In [4]:
# 4. Liệt kê các sản phẩm có tồn kho dưới 10.

cursor.execute("""
SELECT p.name, v.stock, od.quantity
FROM products p
INNER JOIN variants v
    ON p.productid = v.productid
INNER JOIN orderdetails od
    ON v.variantid = od.variantid
WHERE v.stock < 10;
""")

data = cursor.fetchall()
print(data)

(('Váy xòe', 5, 1),)


In [5]:
# 5. Tính tổng doanh thu theo từng ngày đặt hàng.

cursor.execute("""
SELECT o.orderdate, sum(od.quantity * od.price) 
    AS revenue_date 
FROM orders o
INNER JOIN orderdetails od
    ON o.orderid = od.orderid
GROUP BY o.orderdate;
""")

data = cursor.fetchall()
print(data)

((datetime.date(2024, 3, 1), Decimal('1100000.00')), (datetime.date(2024, 3, 2), Decimal('450000.00')))


In [6]:
# 6. Liệt kê các khách hàng đã từng có đơn hàng.

cursor.execute("""
SELECT o.orderid, c.name
FROM orders o
INNER JOIN customers c
    ON c.customerid = o.customerid;
""")

data = cursor.fetchall()
print(data)

((1, 'Trần An'), (2, 'Lê Bình'))


In [7]:
# 7. Liệt kê sản phẩm bán chạy nhất (theo tổng số lượng bán).

cursor.execute("""
SELECT p.name, v.color, v.size, od.quantity
FROM products p
INNER JOIN variants v
    ON p.productid = v.productid
INNER JOIN orderdetails od
    ON v.variantid = od.variantid
ORDER BY od.quantity DESC;
""")

data = cursor.fetchall()
print(data)

(('Áo sơ mi nam', 'Trắng', 'M', 2), ('Quần jeans', 'Xanh', 'M', 1), ('Váy xòe', 'Đỏ', 'S', 1))


In [8]:
# 8. Tính tổng doanh thu theo từng loại danh mục sản phẩm.

cursor.execute("""
SELECT p.name, v.color, v.size, p.price * od.quantity
    AS revenue_per_product
FROM products p
INNER JOIN variants v
    ON p.productid = v.productid
INNER JOIN orderdetails od
    ON v.variantid = od.variantid;
""")

data = cursor.fetchall()
print(data)

(('Áo sơ mi nam', 'Trắng', 'M', Decimal('600000.00')), ('Quần jeans', 'Xanh', 'M', Decimal('500000.00')), ('Váy xòe', 'Đỏ', 'S', Decimal('450000.00')))


In [9]:
# 9. Hiển thị chi tiết của một đơn hàng cụ thể (gồm tên sản phẩm, màu, size, số lượng, đơn giá, tổng dòng).

cursor.execute("""
SELECT p.name, v.color, v.size, v.stock, p.price * od.quantity
    AS total_price
FROM products p
INNER JOIN variants v
    ON p.productid = v.productid
INNER JOIN orderdetails od
    ON v.variantid = od.variantid;
""")

data = cursor.fetchall()
print(data)

(('Áo sơ mi nam', 'Trắng', 'M', 20, Decimal('600000.00')), ('Quần jeans', 'Xanh', 'M', 10, Decimal('500000.00')), ('Váy xòe', 'Đỏ', 'S', 5, Decimal('450000.00')))


In [10]:
# 10. Liệt kê các khách hàng chưa từng thực hiện đơn hàng nào.

cursor.execute("""
SELECT c.name, o.orderdate
FROM customers c
RIGHT JOIN orders o
    ON c.customerid = o.customerid
WHERE c.name IS NULL;
""")

data = cursor.fetchall()
print(data)

()


In [11]:
# 11. Thống kê số lượng khách hàng mới theo từng tháng.

cursor.execute("""
SELECT DATE_FORMAT(createddate, '%m-%y') AS month, count(customerid) AS person_created
FROM customers 
GROUP BY month;
""")

data = cursor.fetchall()
print(data)

(('01-24', 1), ('02-24', 1))


In [13]:
# 12. Thống kê số đơn hàng theo từng trạng thái (đã giao, đã đặt, đã huỷ).

cursor.execute("""
SELECT p.name, v.color, v.size, o.status 
FROM products p
INNER JOIN variants v
    ON p.productid = v.productid
INNER JOIN orderdetails od
    ON od.variantid = v.variantid
INNER JOIN orders o
    ON o.orderid = od.orderid;
""")

data = cursor.fetchall()
print(data)

(('Áo sơ mi nam', 'Trắng', 'M', 'Đã giao'), ('Quần jeans', 'Xanh', 'M', 'Đã giao'), ('Váy xòe', 'Đỏ', 'S', 'Đã đặt'))


In [15]:
# 13. Tính tồn kho trung bình theo từng loại sản phẩm.

cursor.execute("""
SELECT c.name, avg(v.stock)
    AS avg_stock  
FROM categories c
INNER JOIN products p
    ON p.categoryid = c.categoryid
INNER JOIN variants v
    ON p.productid = v.productid
GROUP BY
    c.categoryid;
""")

data = cursor.fetchall()
print(data)

(('Áo', Decimal('17.5000')), ('Quần', Decimal('10.0000')), ('Váy', Decimal('5.0000')))


In [16]:
# Close Connection
cursor.close()
conn.close()